https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/

In [ ]:
!mkdir -p ./data ./raw_data

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import six

In [ ]:
URL = 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap'

In [ ]:
LABELS = [
    'apple', 'bed', 'cat', 'dog', 'eye',
    'fish', 'grass', 'hand', 'ice cream', 'jacket',
]

In [ ]:
for l in LABELS:
    url = '{}/{}.npy'.format(URL, l)
    !wget -P raw_data "$url"

In [ ]:
raw_data = {key: np.load('raw_data/{}.npy'.format(key)) for key in LABELS}

In [ ]:
for k, v in six.iteritems(raw_data):
    print('{:10}: {}'.format(k,len(v)))

In [ ]:
plt.imshow(np.reshape(raw_data['cat'][0], [28, 28]), cmap='gray_r')
plt.show()

In [ ]:
def get_example_proto(image, label):
    return tf.train.Example(features=tf.train.Features(feature={
        'image' : tf.train.Feature(float_list=tf.train.FloatList(value=image)),
        'label' : tf.train.Feature(int64_list=tf.train.Int64List(value=label)),
    })).SerializeToString()

train_data = []
test_data = []
for label_name, value in six.iteritems(raw_data):
    label_index = LABELS.index(label_name)
    print('proccessing label {}: "{}"'.format(label_index, label_name))
    value = np.asarray(value) / 255.
    tr = value[:70000]
    te = value[70000:100000]
    train_data.extend(zip(tr, np.full(70000, label_index)))
    test_data.extend(zip(te, np.full(30000, label_index)))

In [ ]:
np.random.shuffle(train_data)
np.random.shuffle(test_data)

In [ ]:
train_filename = './data/train.tfr'
test_filename  = './data/test.tfr'
tfr_options = tf.python_io.TFRecordOptions(tf.python_io.TFRecordCompressionType.GZIP)
with tf.python_io.TFRecordWriter(train_filename, tfr_options) as train_tfr, \
     tf.python_io.TFRecordWriter(test_filename, tfr_options) as test_tfr:
    for data, label in train_data:
        train_tfr.write(get_example_proto(data, [label]))
    for data, label in test_data:
        test_tfr.write(get_example_proto(data, [label]))